# Welcome to TQ42

## Example of training a generic time series Neural Network

In [ ]:
#Import Packages
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto

from tq42.algorithm import (
    TrainDataProcessingParametersProto,
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto, 
    ActFuncProto,
    GenericMLTrainMetadataProto,
    GenericMLTrainParametersProto,
    Layer,
    PQNLayer,
    ClassicalDenseLayer,
    MLModelType,
    ActivationFunctionLayer,
    ActFuncProto,
    DropoutLayer,
    BatchNormalizationLayer,
    TrainModelInfoProto,
    MLTrainInputsProto,
    AlgorithmProto,
    MeasurementModeProto,
    MeasureProto,
    EntanglingProto,
    DiffMethodProto,
    QubitTypeProto,
    ClassicalLSTMLayer,
    QLSTMLayer,
)

from google.protobuf.json_format import MessageToDict

# Create a client. Select an Organization, Project and Experiment

In [ ]:
# Connect to the cloud service
# https://terraquantum.io
with TQ42Client() as client:
    client.login()

In [ ]:
# list the organizations available to you 
for organization in list_all_organizations(client=client):
    print (organization.id)

In [ ]:
# Select an organization from the list
org_selected = list_all_organizations(client=client)[0]
print (org_selected.id)

In [ ]:
# List the projects within that organization
for project in list_all_projects(client=client, organization_id=org_selected.id):
    print (project.id)

In [ ]:
# Select a project from the list
proj_selected = list_all_projects(client=client, organization_id=org_selected.id)[0]
print(proj_selected.id)

In [ ]:
# List the experiments within that project
for experiment in list_all_experiments(client=client, project_id=proj_selected.id):
    print (experiment.id)

In [ ]:
# Select an experiment from the list
exp_selected = list_all_experiments(client=client, project_id=proj_selected.id)[0]
print(exp_selected.id)

# Build and Train the Neural Network

### Classical Example

In [ ]:
# Build the classical neural network

classical_train_message=MessageToDict(GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        # Choose model type here
        model_type=MLModelType.MLP,
        # Add and customize and customize layers here
        layers=[
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=30, bias=True)),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=45, bias=True)),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=60, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(dropout_layer=DropoutLayer(value=0.5)),
            Layer(batch_normalization_layer=BatchNormalizationLayer()),
            Layer(
                classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=False)
            ),  # Final layer, bias=False
        ],
        num_epochs=5,
        batch_size=128,
        learning_rate=0.05,
        k_fold=1,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model.
            name="ENTER_MODEL_NAME_HERE",
            # Add a brief description to help users understand the purpose or functionality of this trained model.
            description="ADD_DESCRIPTION_HERE",
        ),
        data_processing_parameters=TrainDataProcessingParametersProto(
            input_columns=[0, 1, 2, 3],
            output_columns=[4]
        )
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id="ENTER_DATASET_STORAGE_ID_HERE")
    ),
),preserving_proto_field_name=True)

In [ ]:
# Train the classical neural network

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN, 
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL, 
    parameters=classical_train_message
)

print(run.data)

### Hybrid Quantum Example

In [ ]:
# Build the hybrid quantum neural network

hybrid_quantum_train_message=MessageToDict(GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        # Choose model type here
        model_type=MLModelType.MLP,
        # Add and customize and customize layers here
        layers=[
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=4, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=4, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(
                pqn_layer=PQNLayer(
                    num_qubits=4,
                    depth=7,
                    measurement_mode=MeasurementModeProto.NONE,
                    rotation=MeasureProto.X,
                    entangling=EntanglingProto.BASIC,
                    measure=MeasureProto.Z,
                    diff_method=DiffMethodProto.ADJOINT,
                    qubit_type=QubitTypeProto.LIGHTNING_QUBIT,
                )
            ),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=False)),
        ],
        num_epochs=5,
        k_fold=1,
        batch_size=128,
        learning_rate=0.05,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model.
            name="ENTER_MODEL_NAME_HERE",
            # Add a brief description to help users understand the purpose or functionality of this trained model.
            description="ADD_DESCRIPTION_HERE",
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id="ENTER_DATASET_STORAGE_ID_HERE")
    ),
),preserving_proto_field_name=True)

In [ ]:
# Train the hybrid quantum neural network

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN, 
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL, 
    parameters=hybrid_quantum_train_message
)

print(run.data)